In [1]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input
from tensorflow.keras.models import Model
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle

2023-09-26 12:31:24.737684: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def custom_input_layer(input_shape=(80, 160, 3)):
    inputs = Input(shape=input_shape)
    return inputs

# Define the LaneNet-like model
def LaneNet(input_shape=(80, 160, 3)):
    inputs = custom_input_layer(input_shape)
    
    # Encoder
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    
    # Decoder
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    
    model = Model(inputs, x)
    return model

# Create the LaneNet model with input size (80, 160, 3)
model = LaneNet(input_shape=(80, 160, 3))


In [3]:
    # Load training images
    train_images = pickle.load(open("full_CNN_train.p", "rb" ))

    # Load image labels
    labels = pickle.load(open("full_CNN_labels.p", "rb" ))

    # Make into arrays as the neural network wants these
    train_images = np.array(train_images)
    labels = np.array(labels)

    # Normalize labels - training images get normalized to start in the network
    labels = labels / 255

    # Shuffle images along with their labels, then split into training/validation sets
    train_images, labels = shuffle(train_images, labels)
    # Test size may be 10% or 20%
    X_train, X_val, y_train, y_val = train_test_split(train_images, labels, test_size=0.3)

    # Batch size, epochs and pool size below are all paramaters to fiddle with for optimization
    batch_size = 128
    epochs = 20 
    pool_size = (2, 2)
    input_shape = X_train.shape[1:]

    # Create the neural network
    model = LaneNet(input_shape)

    # Using a generator to help the model use less data
    # Channel shifts help with shadows slightly
    datagen = ImageDataGenerator(channel_shift_range=0.2)
    datagen.fit(X_train)

    # Compiling and training the model
    model.compile(optimizer='adam', loss='mean_squared_error',metrics=['accuracy'])
    model.fit(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=len(X_train)/batch_size,
    epochs=epochs, verbose=1, validation_data=(X_val, y_val))

    # Freeze layers since training is done
    model.trainable = False
    model.compile(optimizer='adam', loss='mean_squared_error',metrics=['accuracy'])

    # Save model architecture and weights
    model.save('Lane_Net_new_model.keras')
    
    #save model architecture in json file
    model_json = model.to_json()
    with open("lane_Net_model.json", "w") as json_file:
        json_file.write(model_json)


Epoch 1/20
69/69 [==============================] - 212s 3s/step - loss: 0.1600 - accuracy: 0.8207 - val_loss: 0.1523 - val_accuracy: 0.8298
Epoch 2/20
69/69 [==============================] - 209s 3s/step - loss: 0.1534 - accuracy: 0.8287 - val_loss: 0.1523 - val_accuracy: 0.8298
Epoch 3/20
69/69 [==============================] - 209s 3s/step - loss: 0.1534 - accuracy: 0.8287 - val_loss: 0.1523 - val_accuracy: 0.8298
Epoch 4/20
69/69 [==============================] - 209s 3s/step - loss: 0.1534 - accuracy: 0.8287 - val_loss: 0.1523 - val_accuracy: 0.8298
Epoch 5/20
69/69 [==============================] - 209s 3s/step - loss: 0.1534 - accuracy: 0.8287 - val_loss: 0.1523 - val_accuracy: 0.8298
Epoch 6/20
69/69 [==============================] - 208s 3s/step - loss: 0.1534 - accuracy: 0.8287 - val_loss: 0.1523 - val_accuracy: 0.8298
Epoch 7/20
69/69 [==============================] - 209s 3s/step - loss: 0.1534 - accuracy: 0.8287 - val_loss: 0.1523 - val_accuracy: 0.8298
Epoch 8/20
69

In [4]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 80, 160, 3)]      0         
                                                                 
 conv2d_4 (Conv2D)           (None, 80, 160, 64)       1792      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 40, 80, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 40, 80, 128)       73856     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 20, 40, 128)       0         
 g2D)                                                            
                                                                 
 up_sampling2d_2 (UpSamplin  (None, 40, 80, 128)       0   

In [7]:
import numpy as np
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score

predicted_labels= model.predict(X_val)

accuracy= accuracy_score(y_val,predicted_labels)
precision= precision_score(y_val,predicted_labels)
recall= recall_score(y_val,predicted_labels)
f1= f1_score(y_val,predicted_labels)

print(f"accuracy:{accuracy:.4f}")
print(f"precision:{precision:.4f}")
print(f"recall:{recall:.4f}")
print(f"F1_score:{f1:.4f}")

120/120 [==============================] - 15s 126ms/step


ValueError: unknown is not supported

In [11]:
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


predicted_labels= model.predict(X_val)

# Convert predicted labels to binary format (0 or 1) using a threshold (e.g., 0.5)
predicted_labels_binary = (predicted_labels > 0.5).astype(int)

# Calculate and plot the confusion matrix
conf_matrix = confusion_matrix(y_val.flatten(), predicted_labels_binary.flatten())

# Plot the confusion matrix
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# Calculate and print accuracy
accuracy = accuracy_score(y_val.flatten(), predicted_labels_binary.flatten())
print("Accuracy:", accuracy)

120/120 [==============================] - 15s 125ms/step


ValueError: Classification metrics can't handle a mix of continuous and binary targets

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Example continuous and binary targets (replace with your actual data)
train_images = np.random.rand(100)  # Continuous values between 0 and 1
binarydnp.random.rand(100) > 0.5).astype(int)  # Binary labels (0 or 1)

# Assuming 'continuous_targets' and 'binary_targets' are your actual targets
# Replace them with your own data

# Convert continuous targets to binary using a threshold
threshold = 0.5  # Adjust the threshold as needed
predicted_binary = (continuous_targets > threshold).astype(int)

# Calculate the confusion matrix for binary classification
conf_matrix_binary = confusion_matrix(binary_targets, predicted_binary)

# Plot the confusion matrix for binary classification
sns.heatmap(conf_matrix_binary, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel('Predicted Labels (Binary)')
plt.ylabel('True Labels (Binary)')
plt.title('Confusion Matrix (Binary Classification)')
plt.show()
